# utils_migrate Tests

> Tests for database migration utilities using SQLite in-memory.

In [ ]:
import tempfile
import shutil
from pathlib import Path
from fastsql import Database
from sqlalchemy import text
from fh_saas.utils_migrate import (
    Migration, parse_migration, discover_migrations,
    MigrationStatus, get_status, apply_migrations,
    DestructiveRollbackError, rollback, print_status
)

## Test Fixtures

In [ ]:
def create_test_migrations(tmpdir: Path) -> Path:
    """Create test migration files in a temp directory."""
    migrations_dir = tmpdir / "migrations"
    migrations_dir.mkdir()
    
    # Migration 001 - Create users table
    (migrations_dir / "001_create_users.sql").write_text("""
-- UP --
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    email TEXT NOT NULL UNIQUE
);

-- DOWN --
DROP TABLE users;
""")
    
    # Migration 002 - Add name column (non-destructive rollback)
    (migrations_dir / "002_add_name.sql").write_text("""
-- UP --
ALTER TABLE users ADD COLUMN name TEXT;

-- DOWN --
ALTER TABLE users DROP COLUMN name;
""")
    
    # Migration 003 - Create posts table
    (migrations_dir / "003_create_posts.sql").write_text("""
-- UP --
CREATE TABLE posts (
    id INTEGER PRIMARY KEY,
    user_id INTEGER REFERENCES users(id),
    title TEXT NOT NULL
);

-- DOWN --
DROP TABLE posts;
""")
    
    return migrations_dir

## Test parse_migration

In [ ]:
def test_parse_migration():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        migration = parse_migration(migrations_dir / "001_create_users.sql")
        
        assert migration.version == 1
        assert migration.name == "create_users"
        assert "CREATE TABLE users" in migration.up_sql
        assert "DROP TABLE users" in migration.down_sql
        assert migration.is_destructive == True  # DROP TABLE
        assert len(migration.checksum) == 16
        
        print("✓ test_parse_migration passed")

test_parse_migration()

✓ test_parse_migration passed


## Test discover_migrations

In [ ]:
def test_discover_migrations():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        migrations = discover_migrations(migrations_dir)
        
        assert len(migrations) == 3
        assert migrations[0].version == 1
        assert migrations[1].version == 2
        assert migrations[2].version == 3
        
        print("✓ test_discover_migrations passed")

test_discover_migrations()

✓ test_discover_migrations passed


## Test get_status

In [ ]:
def test_get_status_fresh_db():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        status = get_status(db, migrations_dir)
        
        assert status.current_version == 0
        assert status.pending_count == 3
        assert len(status.applied_migrations) == 0
        assert len(status.pending_migrations) == 3
        
        print("✓ test_get_status_fresh_db passed")

test_get_status_fresh_db()

✓ test_get_status_fresh_db passed


## Test apply_migrations

In [ ]:
def test_apply_migrations():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        
        # Apply all migrations
        applied = apply_migrations(db, migrations_dir)
        
        assert len(applied) == 3
        
        # Verify tables exist
        tables = db.execute(text("SELECT name FROM sqlite_master WHERE type='table'")).fetchall()
        table_names = [t[0] for t in tables]
        assert "users" in table_names
        assert "posts" in table_names
        assert "_migrations" in table_names
        
        # Verify status
        status = get_status(db, migrations_dir)
        assert status.current_version == 3
        assert status.pending_count == 0
        
        print("✓ test_apply_migrations passed")

test_apply_migrations()

✓ test_apply_migrations passed


In [ ]:
def test_apply_migrations_dry_run():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        
        # Dry run - should not apply
        pending = apply_migrations(db, migrations_dir, dry_run=True)
        
        assert len(pending) == 3
        
        # Verify nothing was applied
        status = get_status(db, migrations_dir)
        assert status.pending_count == 3
        
        print("✓ test_apply_migrations_dry_run passed")

test_apply_migrations_dry_run()

✓ test_apply_migrations_dry_run passed


In [ ]:
def test_apply_migrations_target_version():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        
        # Apply only up to version 2
        applied = apply_migrations(db, migrations_dir, target_version=2)
        
        assert len(applied) == 2
        
        status = get_status(db, migrations_dir)
        assert status.current_version == 2
        assert status.pending_count == 1
        
        print("✓ test_apply_migrations_target_version passed")

test_apply_migrations_target_version()

✓ test_apply_migrations_target_version passed


## Test rollback

In [ ]:
def test_rollback_requires_force_for_destructive():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        apply_migrations(db, migrations_dir)
        
        # Rollback without force should raise error
        try:
            rollback(db, migrations_dir, steps=1, force=False)
            assert False, "Should have raised DestructiveRollbackError"
        except DestructiveRollbackError as e:
            assert "003_create_posts" in str(e)
            assert "force=True" in str(e)
            print("✓ test_rollback_requires_force_for_destructive passed")

test_rollback_requires_force_for_destructive()

✓ test_rollback_requires_force_for_destructive passed


In [ ]:
def test_rollback_with_force():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        apply_migrations(db, migrations_dir)
        
        # Rollback with force
        rolled_back = rollback(db, migrations_dir, steps=1, force=True)
        
        assert len(rolled_back) == 1
        assert rolled_back[0].version == 3
        
        # Verify posts table dropped
        tables = db.execute(text("SELECT name FROM sqlite_master WHERE type='table'")).fetchall()
        table_names = [t[0] for t in tables]
        assert "posts" not in table_names
        assert "users" in table_names  # Still exists
        
        status = get_status(db, migrations_dir)
        assert status.current_version == 2
        
        print("✓ test_rollback_with_force passed")

test_rollback_with_force()

✓ test_rollback_with_force passed


In [ ]:
def test_rollback_to_target_version():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        apply_migrations(db, migrations_dir)
        
        # Rollback to version 1 (should rollback 3 and 2)
        rolled_back = rollback(db, migrations_dir, target_version=1, force=True)
        
        assert len(rolled_back) == 2
        
        status = get_status(db, migrations_dir)
        assert status.current_version == 1
        assert status.pending_count == 2
        
        print("✓ test_rollback_to_target_version passed")

test_rollback_to_target_version()

✓ test_rollback_to_target_version passed


In [ ]:
def test_rollback_dry_run():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        apply_migrations(db, migrations_dir)
        
        # Dry run rollback
        to_rollback = rollback(db, migrations_dir, steps=1, force=True, dry_run=True)
        
        assert len(to_rollback) == 1
        
        # Verify nothing was actually rolled back
        status = get_status(db, migrations_dir)
        assert status.current_version == 3
        
        print("✓ test_rollback_dry_run passed")

test_rollback_dry_run()

✓ test_rollback_dry_run passed


## Test is_destructive

In [ ]:
def test_is_destructive_detection():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        migrations = discover_migrations(migrations_dir)
        
        # 001 has DROP TABLE - destructive
        assert migrations[0].is_destructive == True
        
        # 002 has DROP COLUMN - destructive  
        assert migrations[1].is_destructive == True
        
        # 003 has DROP TABLE - destructive
        assert migrations[2].is_destructive == True
        
        print("✓ test_is_destructive_detection passed")

test_is_destructive_detection()

✓ test_is_destructive_detection passed


In [ ]:
def test_non_destructive_rollback():
    """Test that non-destructive rollbacks don't require force."""
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = tmppath / "migrations"
        migrations_dir.mkdir()
        
        # Create non-destructive migration (CREATE INDEX / DROP INDEX is non-destructive)
        (migrations_dir / "001_create_idx.sql").write_text("""
-- UP --
CREATE TABLE test (id INTEGER);

-- DOWN --
-- Safe rollback: just comment
SELECT 1;
""")
        
        db = Database("sqlite:///:memory:")
        apply_migrations(db, migrations_dir)
        
        # Should not require force for non-destructive
        rolled_back = rollback(db, migrations_dir, steps=1, force=False)
        
        assert len(rolled_back) == 1
        print("✓ test_non_destructive_rollback passed")

test_non_destructive_rollback()

✓ test_non_destructive_rollback passed


## Test print_status

In [ ]:
def test_print_status():
    with tempfile.TemporaryDirectory() as tmpdir:
        tmppath = Path(tmpdir)
        migrations_dir = create_test_migrations(tmppath)
        
        db = Database("sqlite:///:memory:")
        apply_migrations(db, migrations_dir, target_version=2)
        
        print("\n--- print_status output ---")
        print_status(db, migrations_dir)
        print("--- end ---\n")
        
        print("✓ test_print_status passed")

test_print_status()


--- print_status output ---
Migration Status
Current version: 002
Pending: 1 migration(s)

Applied:
  001_create_users (2026-01-30)
  002_add_name (2026-01-30)

Pending:
  003_create_posts
--- end ---

✓ test_print_status passed


## Run All Tests

In [ ]:
print("\n" + "="*50)
print("All tests passed! ✓")
print("="*50)


All tests passed! ✓
